In [3]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import selenium.common.exceptions
from time import sleep
import csv

class mevzuat:
    ELEMENT_LOCATING_TIMEOUT = 60  # in seconds
    
    def __init__(self):
        self.init_driver()
        self.new_file()
        self.bring_legislations()
        while True:
            firstTextElem = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[1]/td[2]/a/div[1]')))
            firstText = firstTextElem.get_attribute("innerHTML")
                
            i = 1
            while i <= 100:
                self.select_legislation(i)
                self.change_frame()
                self.write_data_to_csv()
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])
                i += 1
            nextPageButton = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#DataTables_Table_0_paginate > ul > li.paginate_button.page-item.active + li > a')))  
            self.safe_click(nextPageButton)
                
            WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until_not(EC.text_to_be_present_in_element_attribute((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[1]/td[2]/a/div[1]'), "innerHTML", firstText))
                
            self.wait_until_page_fully_loaded()
                
    def init_driver(self):
        # options to open driver
        options = Options()
        options.add_argument('start-maximized')
        options.add_argument('disable-infobars')
        options.add_argument('--disable-extensions')
        options.add_argument('--headless')
        
        # create web driver element
        self.driver = webdriver.Chrome(chrome_options=options, executable_path='/Users/emircan/Desktop/Lexlata/Mevzuat Scrape/chromedriver')
        self.URL = 'https://www.mevzuat.gov.tr/#kanunlar'
        self.driver.get(self.URL)
        
    def bring_legislations(self):
        # click on 'ARA' button
        searchElement = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.ID, 'btnSearch')))
        searchElement.click()
        
        self.wait_until_page_fully_loaded()
        
        self.set_listing()
        
    def wait_until_page_fully_loaded(self):
        WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[1]/td[2]/a/div[1]')))
    
    def safe_click(self, element):
        # safely clicks the given element, meaning loading screen won't get front of the element to be clicked
        try:
            element.click()
        except (selenium.common.exceptions.ElementClickInterceptedException, selenium.common.exceptions.StaleElementReferenceException) as e:
            try:
                sleep(2)
                self.safe_click(element)
            except selenium.common.exceptions.StaleElementReferenceException:
                pass
    
    def set_listing(self):
        # change the listing to 100
        listingSelect = Select(WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '//*[@id="DataTables_Table_0_length"]/label/select'))))
        listingSelect.select_by_value("100")
        
        try:
            WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div/div/div/div/div[5]/div/div/div/div[2]/div/table/tbody/tr[100]/td[2]/a/div[1]')))
        except selenium.common.exceptions.TimeoutException:
            pass
        
    def select_legislation(self, i):
        content_xpath = f'//*[@id="DataTables_Table_0"]/tbody/tr[{i}]/td[2]/a'
        link = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.XPATH, content_xpath)))
        href = link.get_attribute('href')
        self.driver.execute_script("window.open(arguments[0]);", href)
        self.driver.switch_to.window(self.driver.window_handles[1])
        
    def change_frame(self):
        iframe = WebDriverWait(self.driver, mevzuat.ELEMENT_LOCATING_TIMEOUT).until(EC.presence_of_element_located((By.TAG_NAME, 'iframe')))
        self.driver.switch_to.frame(iframe)
        
    def new_file(self):
        newfile = open('mevzuat_new.txt', 'w', encoding='utf-8')
        
        header = ["data_text"]
        self.writer = csv.writer(newfile)
        self.writer.writerow(header)
        
    def write_data_to_csv(self):   
        textSegmentElements = self.driver.find_elements(By.XPATH, '/html/body/div/child::*')
        totalText = ''
        for textSegmentElement in textSegmentElements:
            textSegment = textSegmentElement.text.replace('\n', ' ').strip()
            if len(textSegment) == 0 or textSegment.isdigit():
                continue
            if textSegment[-1] != '.':
                textSegment += '.'
            textSegment += ' '
            totalText += textSegment
        
        self.writer.writerow([totalText])
        
            
mevzuat()

/var/folders/v0/w3_rg1yn73q9cjs3q6kf4v400000gn/T/ipykernel_3491/977128295.py:49: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(chrome_options=options, executable_path='/Users/emircan/Desktop/Lexlata/Mevzuat Scrape/chromedriver')
/var/folders/v0/w3_rg1yn73q9cjs3q6kf4v400000gn/T/ipykernel_3491/977128295.py:49: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(chrome_options=options, executable_path='/Users/emircan/Desktop/Lexlata/Mevzuat Scrape/chromedriver')


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000102b81d60 chromedriver + 3792224
1   chromedriver                        0x0000000102b18874 chromedriver + 3360884
2   chromedriver                        0x0000000102819184 chromedriver + 217476
3   chromedriver                        0x0000000102848f5c chromedriver + 413532
4   chromedriver                        0x0000000102871b2c chromedriver + 580396
5   chromedriver                        0x000000010283e3e4 chromedriver + 369636
6   chromedriver                        0x0000000102b57234 chromedriver + 3617332
7   chromedriver                        0x0000000102b5b6c4 chromedriver + 3634884
8   chromedriver                        0x0000000102b5fcb8 chromedriver + 3652792
9   chromedriver                        0x0000000102b5c15c chromedriver + 3637596
10  chromedriver                        0x0000000102b3a7f8 chromedriver + 3500024
11  chromedriver                        0x0000000102b73c3c chr

In [8]:
import pandas as pd
x = pd.read_csv('/Users/emircan/Desktop/untitled folder/mevzuat_new.csv')

In [9]:
x

,data_text
0,DEVLET MEMURLARI KANUNU İLE BAZI KANUNLARDA VE...
1,İSTANBUL FİNANS MERKEZİ KANUNU. Kanun Numarası...
2,SPOR KULÜPLERİ VE SPOR FEDERASYONLARI KANUNU. ...
3,NÜKLEER DÜZENLEME KANUNU. Kanun Numarası ...
4,ÖĞRETMENLİK MESLEK KANUNU. Kanun Numarası ...
...,...
886,CEVAMİİ ŞERİFE VESAİR MÜESSESATI HAYRİYENİN. T...
887,MENAFİİ UMUMİYEYE MÜTEALLİK İMTİYAZAT. HAKKIND...
888,SALTANATI SENİYYE ŞEHBENDERLERİNE DAİR. NİZAMN...
889,126-7. ECANİBİN HAKKI İSTİMLAKİ KANUNUNUN BİRİ...
